<a href="https://colab.research.google.com/github/JjKkhld/parsers/blob/main/Parser_otomoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries and user Agent




In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from os import replace

userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.3'

# Parser - Run to Update a Data

## Functions

In [ ]:
# func of getting all page's links
def scraping_all_pages(limit): # range is a list of all positions o the page - find_all
  url_page = 'https://www.otomoto.pl/osobowe/szczecin?search%5Bdist%5D=50&search%5Border%5D=created_at%3Adesc&page='
  r = requests.get(url_page, headers={'User-Agent': userAgent})
  soup = BeautifulSoup(r.text, 'html')

  # DEFINITION OF RESULT LIST
  all_pages = []

  # getting pages
  last_page = int(soup.find_all('a', {'class':'ooa-g4wbjr e1y5xfcl0'})[3].find('span').text)

  # creating the links
  for el in range(2, last_page):
    all_pages.append(url_page + str(el))

  scraping_first_page = scraping_page(url_page)

  if limit == 1:
    return scraping_first_page
  elif limit == 'all':
    for i in all_pages:
      scraping_first_page += scraping_page(i)
  else:
    for i in all_pages[1:limit - 1]:
      scraping_first_page += scraping_page(i)
  return scraping_first_page

In [ ]:
# function of scraping one whole page
def scraping_page(url): # range is a list of all positions o the page - find_all

  r = requests.get(url, headers={'User-Agent': userAgent})
  soup = BeautifulSoup(r.text, 'html')

  all_pos = soup.find_all('article', {'class': 'ooa-1yux8sr e15xeixv0'})


  # definition of res lists
  prices = []
  links = []
  names = []
  kms = []
  types = []
  transm = []
  years = []
  infos = []
  locations = []
  offers = []

  # checking each position on the page
  for el in all_pos:

    # model name
    name_1 = el.find('h2', {'class': 'e1n1d04s0 ooa-1kyyooz er34gjf0'})
    if name_1:
      name = name_1.find('a')
      if name:
        names.append(name.text)
      else:
        names.append('Lack of name')
    else:
      names.append('Lack of name')

    # model price
    price = el.find('h3', {'class':'e6r213i1 ooa-1n2paoq er34gjf0'})
    if price:
      prices.append(price.text)
    else:
      prices.append('0')

    # link
    link = el.find('h2', {'class': 'e1n1d04s0 ooa-1kyyooz er34gjf0'})
    if link:
      link_data = link.find('a', href=True)['href']
      if link_data:
        links.append(link_data)
      else:
        links.append('Lack of link')
    else:
      links.append('Lack of link')

    if len(el.find_all('dd', {'class': 'ooa-1omlbtp ey6oyue11'})) == 4:

      #kilomiters
      km_data = el.find_all('dd', {'class': 'ooa-1omlbtp ey6oyue11'})[0]
      if km_data:
        kms.append(km_data.text)
      else:
        kms.append(0)

      # engine type
      type_data = el.find_all('dd', {'class': 'ooa-1omlbtp ey6oyue11'})[1]
      if km_data:
        types.append(type_data.text)
      else:
        types.append('None')

      # transmission
      transm_data = el.find_all('dd', {'class': 'ooa-1omlbtp ey6oyue11'})[2]
      if transm_data:
        transm.append(transm_data.text)
      else:
        transm.append('None')

      # year
      year = el.find_all('dd', {'class': 'ooa-1omlbtp ey6oyue11'})[3]
      if year:
        years.append(year.text)
      else:
        years.append('None')
    else:
      pass

    # other info - to further processing
    info = el.find('p', {'class': 'ey6oyue8 ooa-1tku07r er34gjf0'})
    if info:
      clean_info = info.text
      infos.append(clean_info)
    else:
      infos.append('Lack of info')

    # location
    loc_1 = el.find('dl', {'class': 'ooa-1o0axny ey6oyue12'})
    if loc_1:
      loc = loc_1.find_all('p', {'class': 'ooa-gmxnzj'})[0]
      if loc:
        locations.append(loc.text)

      offer_info = el.find('article', {'class': 'ooa-12g3tpj ebwza7n1'}).text
      if 'Zobacz ogłoszenia' in offer_info:
        offers.append('Agent')
      else:
        offers.append('Private saller')
    else:
      offers.append('Private saller')

  return list(zip(names, prices, links, kms, types, \
                    transm, years, infos, locations, offers))

## Parsering and Creating a DataFrame

In [ ]:
data = scraping_all_pages('all')

# print(data)

column_names = ['Name of Model', 'Price', 'Link', 'Kilometers', \
                'Type', 'Transmission', 'Year', 'Information', 'Location', 'Offer Type']
db = pd.DataFrame(data, columns=column_names)

In [ ]:
with open('/content/drive/MyDrive/DataFrames/Parsers/auto_Szczecin_alldata.csv', \
          'w', encoding='utf-8-sig') as file:
    db.to_csv(file, index=False)

# Processing a dataframe

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DataFrames/Parsers/auto_Szczecin_alldata.csv')

## Total records, define numeric & non-num columns

In [ ]:
print("Number of rows: {}".format(data.shape[0]))
print("Number of columns: {}".format(data.shape[1]))

print("Columns in our data: {}".format(data.columns.values))

Number of rows: 6073
Number of columns: 10
Columns in our data: ['Name of Model' 'Price' 'Link' 'Kilometers' 'Type' 'Transmission' 'Year'
 'Information' 'Location' 'Offer Type']


**'Name_of_col' - dtype is expected**
* 'Brand' - str (new column)
* 'Model' - str
* 'Price' - float
* 'Link' - str
* 'Kilometers' - float
* 'Type' - str
* 'Transmission' - str
* 'Year' - int
* 'Information' - str
* 'Location' - str
* 'Offer type' - str

Here we can check if dtypes are appropriate

In [ ]:
# select numeric columns
data_num = data.select_dtypes(include=[np.number])
numeric_cols = data_num.columns.values
print("There are numeric columns: {}".format(numeric_cols))

# select non_numeric columns
data_nonnum = data.select_dtypes(exclude=[np.number])
nonnumeric_cols = data_nonnum.columns.values
print("There are non-numeric columns: {}".format(nonnumeric_cols))

There are numeric columns: ['Year']
There are non-numeric columns: ['Name of Model' 'Price' 'Link' 'Kilometers' 'Type' 'Transmission'
 'Information' 'Location' 'Offer Type']


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6073 entries, 0 to 6072
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name of Model  6073 non-null   object
 1   Price          6073 non-null   object
 2   Link           6073 non-null   object
 3   Kilometers     6073 non-null   object
 4   Type           6073 non-null   object
 5   Transmission   6073 non-null   object
 6   Year           6073 non-null   int64 
 7   Information    6073 non-null   object
 8   Location       6073 non-null   object
 9   Offer Type     6073 non-null   object
dtypes: int64(1), object(9)
memory usage: 474.6+ KB


## Changing dtypes

In [ ]:
data['Price'] = data['Price'].apply(lambda row: str(row).replace(' ', ''))
data['Price'] = data['Price'].astype('int')

data['Kilometers'] = data['Kilometers'].apply(lambda row: str(row).replace(' ', '')) \
                                       .apply(lambda row: str(row).replace('km', ''))
data['Kilometers'] = data['Kilometers'].astype('int')

## Processing with new columns and finish of changing dtypes

In [ ]:
data['Name of Model'] = data['Name of Model'].apply(lambda row: str(row).split())


data['Brand'] = data['Name of Model'].apply(lambda row: row[0]) \
                                     .apply(lambda row: 'Alfa Romeo' if row == 'Alfa' else row)
data['Model'] = data['Name of Model'].apply(lambda row: ' '.join(row[1:]))

data = data.drop('Name of Model', axis=1)

In [ ]:
data['Information'] = data['Information'].apply(lambda row: str(row).split(' • '))

rows_to_drop = data.loc[data['Information'].apply(lambda x: len(x)) < 2].index
data = data.drop(rows_to_drop, axis=0)

In [ ]:
data.head(2)

,Price,Link,Kilometers,Type,Transmission,Year,Information,Location,Offer Type,Brand,Model
0,15000,https://www.otomoto.pl/osobowe/oferta/volkswag...,272000,Diesel,Manualna,2007,"[1 896 cm3, 84 KM, VW T5 używany gotowy do pra...",Szczecin (Zachodniopomorskie),Private saller,Volkswagen,Transporter T5 7JD162/WF2/ZP1
1,72000,https://www.otomoto.pl/osobowe/oferta/opel-gra...,82000,Diesel,Manualna,2019,"[1 499 cm3, 130 KM, Opel Grandland X 1.5]",Wapnica (Zachodniopomorskie),Private saller,Opel,Grandland X 1.5 CDTI Elegance Business Pack S&S


In [ ]:
def checking_data_cap(row):
  for el in row:
    if ' cm3' in el and len(el.split(' ')) < 5:
      return el
    else:
      continue

def checking_data_pow(row):
  for el in row:
    if ' KM' in el and len(el.split(' ')) < 3:
      return el
    else:
      continue

In [ ]:
data['Capacity[cm3]'] = data['Information'].apply(checking_data_cap) \
                                           .fillna(0)
data['Power[KM]'] = data['Information'].apply(checking_data_pow) \
                                       .fillna(0)

data = data.drop('Information', axis=1)

In [ ]:
columns = ['Brand', 'Model', 'Price', 'Link', 'Kilometers', 'Type', 'Transmission', \
           'Year', 'Capacity[cm3]', 'Power[KM]', 'Location', 'Offer Type']

data = data[columns]

In [ ]:
# changing dtypes of the new columns

data['Capacity[cm3]'] = data['Capacity[cm3]'].apply(lambda row: str(row).replace(' ', '')) \
                     .apply(lambda row: str(row).replace ('cm3', '')).astype('float')

data['Power[KM]'] = data['Power[KM]'].apply(lambda row: str(row).replace(' ', '')) \
                 .apply(lambda row: str(row).replace('KM', '')).astype('float')

In [ ]:
data['Location'] = data['Location'].apply(lambda row: str(row).replace(' (Zachodniopomorskie)', ''))

# Saving final version

In [ ]:
with open('/content/drive/MyDrive/DataFrames/Parsers/auto_Szczecin_alldata.csv', \
          'w', encoding='utf-8-sig') as file:
    data.to_csv(file, index=False)

In [ ]:
db.head(5)

,Name of Model,Price,Link,Kilometers,Type,Transmission,Year,Information,Location,Offer Type
0,Volkswagen Transporter T5 7JD162/WF2/ZP1,15 000,https://www.otomoto.pl/osobowe/oferta/volkswag...,272 000 km,Diesel,Manualna,2007,1 896 cm3 • 84 KM • VW T5 używany gotowy do pr...,Szczecin (Zachodniopomorskie),Private saller
1,Opel Grandland X 1.5 CDTI Elegance Business Pa...,72 000,https://www.otomoto.pl/osobowe/oferta/opel-gra...,82 000 km,Diesel,Manualna,2019,1 499 cm3 • 130 KM • Opel Grandland X 1.5,Wapnica (Zachodniopomorskie),Private saller
2,Kia Stinger 3.3 T-GDI V6 GT Prestige Line AWD,132 000,https://www.otomoto.pl/osobowe/oferta/kia-stin...,26 200 km,Benzyna,Automatyczna,2020,3 342 cm3 • 366 KM • kia stinger gt2 3.3 awd,Szczecin (Zachodniopomorskie),Private saller
3,Volkswagen Golf,112 650,https://www.otomoto.pl/osobowe/oferta/volkswag...,1 km,Benzyna,Manualna,2024,"1 498 cm3 • 150 KM • LIFE, 1.5 TSI 150KM Manua...",Szczecin (Zachodniopomorskie),Agent
4,Kia Niro,85 000,https://www.otomoto.pl/osobowe/oferta/kia-niro...,76 700 km,Hybryda,Automatyczna,2020,1 580 cm3 • 105 KM • KIA Niro,Szczecin (Zachodniopomorskie),Private saller
